In [1]:
# determine which gpu to use
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2022-08-03 17:01:19.962809: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-03 17:01:19.963468: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
# image size becauase we are using vgg16 and vgg exoects size to be 224*224 but we can also give sizes on our own
IMAGE_SIZE=[224,224]
train_path='Cotton Disease/train'
valid_path='Cotton Disease/test'

In [4]:
# here we are using imagenet weight ,import vgg16 libraries adding prprocessing layer ,include_top means we are removing first and the last layer
vgg16=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

2022-08-03 17:01:19.992805: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [5]:
for layer in vgg16.layers:
    layer.trainable=False
#     here we dont have to train the weights as they are already been trained

In [6]:
folder=glob('Cotton Disease/train/*')

In [7]:
folder

['Cotton Disease/train/diseased cotton plant',
 'Cotton Disease/train/diseased cotton leaf',
 'Cotton Disease/train/fresh cotton leaf',
 'Cotton Disease/train/fresh cotton plant']

In [8]:
# adding flatten layer
x=Flatten()(vgg16.output)

In [9]:
len(folder)

4

In [10]:
# len folder means we want 4 outputs
prediction=Dense(len(folder),activation='softmax')(x)
# model object
model=Model(inputs=vgg16.input,outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
model.compile(
loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
# data agumentation is not done to trainnig data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
# we have to mage sure that image generated should be of same size ,we are genraaly data  in train ,test
training_set=train_datagen.flow_from_directory('Cotton Disease/train',target_size=(224,224),batch_size=32,class_mode='categorical')


Found 1951 images belonging to 4 classes.


In [15]:
test_set=test_datagen.flow_from_directory('Cotton Disease/test',target_size=(224,224),batch_size=32,class_mode='categorical')



Found 106 images belonging to 4 classes.


In [16]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
#     The Steps per epoch denote the number of batches to be selected for one epoch. 
#     the validation step is a single backpropagation performed on the validation data. 
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
 4/61 [>.............................] - ETA: 57:38 - loss: 2.0521 - accuracy: 0.3984

KeyboardInterrupt: 